In [1]:
%matplotlib inline
from pandas import Series , DataFrame
import pandas as pd , numpy as np

import matplotlib.pyplot as plt

In [2]:
data = DataFrame(pd.read_csv('C://users/grlurton/documents/dhis_kenya/data_kenya.csv'))
data_categories = DataFrame(pd.read_csv('C://users/grlurton/documents/dhis_kenya/data_categories.csv'))
data_elements = DataFrame(pd.read_csv('C://users/grlurton/documents/dhis_kenya/data_elements.csv'))
org_units = DataFrame(pd.read_csv('C://users/grlurton/documents/dhis_kenya/org_units.csv'))
org_units_groups = DataFrame(pd.read_csv('C://users/grlurton/documents/dhis_kenya/org_units_groups.csv'))

# reports that organization units are supposed to make
org_units_report = DataFrame(pd.read_csv('C://users/grlurton/documents/dhis_kenya/org_units_data_sets.csv'))
print 'N of data values' , len(data)

N of data values 49809663


For simplicity, at this stage, we only select data from reports of form MOH. 

In [3]:
reports_activity = ['oEfGJAQaARj' , 'GVGt4o7VkHk' , 'WtgleFiaJqO' , 'KbrFbt3w4bp', 
                    'b8k5cUrvVqr' , 'sfiNZ7atsK8','NLhjWzDVx4G','ovtKPo15xAg' , 'TXmr8u5rabj', 'DWKwKC6dnoB']

reports_conflicting = ['lPykYxJeLKM' , 'cvyzaRp8OlE'] #These reports have the same data elements so hard to make completeness vs 0

reports_hiv = ['kAofV66isvC' , 'yrYwif6R6sH' , 'GGgrU5QkjVs' , 'NJHaY8wlURg' , 'UeBJcYEoHeA' , 'IH4pYzRqTSE']

data_elements_list = data_elements[data_elements.datasets_ID.isin(reports_activity)]
data_elements_hiv = data_elements[data_elements.datasets_ID.isin(reports_hiv)]
data_weekly = data_elements[data_elements.datasets_ID == 'zJrL4HQqYmD']
print 'data keep :' , len(data_elements_list) , 'indicators ; data hiv :' , len(data_elements_hiv) , 'indicators'
print 'weekly data :' , len(data_weekly), 'indicators'

data keep : 794 indicators ; data hiv : 154 indicators
weekly data : 29 indicators


In [4]:
data_moh = data[data.data_element_ID.isin(data_elements_list.data_element_ID)]
data_moh = data_moh[(data_moh['period'] != '2014AprilS1')]
data_moh = data_moh.drop(['Unnamed: 0' , 'last_update'] , axis = 1)
data_moh = data_moh[data_moh.data_element_ID != 'uxYaacfgY22'] # we don't care about the date the report was received


data_weekly = data[data.data_element_ID.isin(data_weekly.data_element_ID)]
data_weekly = data_weekly.drop(['Unnamed: 0' , 'last_update'] , axis = 1)

print len(data_moh)
print len(data_weekly)

20710483
141343


# Measuring completeness

## Making a full dataset

We first make a full dataset that includes explicitely all missing values. To do this we first put the data in wide format to create all the NA values. We then collapse select only reports due by each facility

In [5]:
#'oEfGJAQaARj'
reports_activity = ['GVGt4o7VkHk' , 'WtgleFiaJqO' , 'KbrFbt3w4bp', #'b8k5cUrvVqr' , 
                    'sfiNZ7atsK8','NLhjWzDVx4G','ovtKPo15xAg' , 'TXmr8u5rabj', 'DWKwKC6dnoB']

## On va voir ce qui dans 'b8k5cUrvVqr' 

moh_elements = data_elements[data_elements.datasets_ID.isin(reports_activity)]

data_moh_merged = pd.merge(data_moh, moh_elements, left_on='data_element_ID', right_on='data_element_ID', 
                                     how='inner')

data_moh_merged.loc[((data_moh_merged.datasets_ID == 'GVGt4o7VkHk') & 
                            (data_moh_merged.category == 'lHl2rmXpHse')),'datasets_ID'] = 'oEfGJAQaARj'

data_moh_merged.loc[(data_moh_merged.datasets_ID == 'oEfGJAQaARj'),'datasets_name'] = "Outpatient summary < 5 years"

data_moh_merged = data_moh_merged.drop(['data_element_url' , 'datasets_name' , 'data_element_name','Unnamed: 0'] , axis = 1)

In [6]:
# First a very wide melt to get complete missing values in time
data_moh_wide = data_moh_merged.set_index(['datasets_ID',  
                                      'data_element_ID','category' ,'org_unit_ID', 'period']).unstack(['org_unit_ID','period']) 

In [7]:
data_moh_wide2 = data_moh_wide.reindex()
#,'period',org_unit_ID'
#help(data_moh_wide.reindex) 

In [8]:
data_moh_wide2.sortlevel(axis=0,inplace=True,sort_remaining=True)
data_moh_wide2.sortlevel(axis=1,inplace=True,sort_remaining=True)

In [9]:
# Functions to zeroify relevant mixing values

#idx = pd.IndexSlice
#nbr_na = []
#ncol = []

def how_much_null(data):
    out = sum(pd.isnull(data))
    return out

def zeroify(data):
    how_much = how_much_null(data)
    total_length  = len(data)
    if how_much < total_length:
        data[pd.isnull(data)] = 0
    return data

In [11]:
## Faire une liste de tout mes facility / months pourlesquels j'ai au moins une value
## Merger sur data complete pour garder juste les données de ces facilités / mois
## fillna là dessus pour changer les données en 0.

In [10]:
df1 = data_moh_wide2

In [11]:
periods = data_moh['period'].unique()
dataSets = data_moh_wide2.index.levels[0]

for report in dataSets:
    print 'Report' , report,  len(data_moh_wide2.loc[report])
    moh_reindex_sub = data_moh_wide2.loc[(report,slice(None),slice(None)),(slice(None) , slice(None) , slice(None))]
    print "     Let's Zeroify !!"
    data_try = moh_reindex_sub.apply(zeroify)
    print "         Creating data frame"
    df1.loc[(report,slice(None),slice(None)),(slice(None) , slice(None) , slice(None))] = data_try

Report DWKwKC6dnoB 240
     Let's Zeroify !!
         Creating data frame
Report GVGt4o7VkHk 52
     Let's Zeroify !!
         Creating data frame
Report KbrFbt3w4bp 54
     Let's Zeroify !!
         Creating data frame
Report NLhjWzDVx4G 279
     Let's Zeroify !!
         Creating data frame
Report TXmr8u5rabj 48
     Let's Zeroify !!
         Creating data frame
Report WtgleFiaJqO 104
     Let's Zeroify !!
         Creating data frame
Report oEfGJAQaARj 47
     Let's Zeroify !!
         Creating data frame
Report ovtKPo15xAg 42
     Let's Zeroify !!
         Creating data frame
Report sfiNZ7atsK8 95
     Let's Zeroify !!
         Creating data frame


In [17]:
df1.to_csv('C://users/grlurton/desktop/out_py.csv')

In [ ]:
samp = range(0,10000)

In [ ]:
moh_reindex2 = moh_reindex.loc[samp]

In [ ]:
wid = data_moh_wide.stack(dropna=False)

In [ ]:
#data_full = df1
#data_full.columns = [' '.join(col).strip() for col in data_full.columns.values]
#data_full.columns = data_full.columns.str[6:29]

# Then we stack on org_unit_ID, so we have all period for all org_unit, including empty ones
#data_moh_full = data_moh_wide.stack(level=['org_unit_ID'] , dropna = False)

#print data_moh_wide.head()

In [ ]:
#data_moh_wide_2 = data_moh_wide.stack(level=['org_unit_ID','data_element_ID'] , dropna = False)
#data_moh_wide_2.reset_index(inplace=True)

In [ ]:
#data_moh_wide_2.reset_index(inplace=True)
#data_moh_wide_2.uniqueid = data_moh_wide_2["org_unit_ID"].str.cat(data_moh_wide_2['data_element_ID'], sep=' ' )
#ID = data_elements_requirement["org_unit_ID"].str.cat(data_elements_requirement['data_element_ID'], sep=' ' )

In [ ]:
data_elements_requirement = pd.merge(org_units_report, df1 , left_on='dataset_ID', right_on='datasets_ID', 
                                     how='inner')

In [ ]:
#col = data_moh_wide.columns
sec_data = data_moh_wide
#for fac in col.levels[1].tolist():
    #if not (fac in data_elements_requirement["org_unit_ID"].unique()) :
        #sec_data = sec_data.drop(level = fac , axis = 1)
        #print 'Dropped' , fac
        
        #indicators = data_elements_requirement[data_elements_requirement["org_unit_ID"] == fac]
        
        #indicators[]
        #print 'ah bah non !'

In [ ]:
idx = pd.IndexSlice
#for fac in data_elements_requirement["org_unit_ID"].unique() :
#    if fac in col.levels[1] :
        #print data_moh_wide.loc[slice(None),(slice(None),fac,slice(None),slice(None))]

In [ ]:
#data_moh_wide.loc[slice(None),(slice(None),'A05h9ICwmHK','BwROYFIiXKG','SeDaCnHi3m5')]

In [ ]:
test = data_full.columns

test2 = test[test.isin(ID)].tolist()

print len(test2)

#print test2

data_ok = data_full[test2]

print len(data_full.columns)

#print data_full.head()

#print len(data_ok.columns) , 'm' , len(data_full.columns)

## Describing missingness

We first compute how much a orgunit should report on a monthly basis

In [ ]:
def get_requirement_orgunit(data_element_list , report_requirement):
    """
    Function that takes a list of data elements and the reporting requirement of facilities and returns a merge data frame 
    representing a facility level requirement of different data elements
    """
    data_elements_requirement = pd.merge(report_requirement, data_element_list, left_on='dataset_ID', right_on='datasets_ID', 
                                     how='inner')
    data_elements_numbers = DataFrame(data_elements_requirement['org_unit_ID'].value_counts())
    data_elements_numbers.columns = ['required_data_elements']
    return data_elements_numbers

def get_requirement_data_element(data_element_list , report_requirement):
    """
    Function that takes a list of data elements and the reporting requirement of facilities and returns a merge data frame 
    representing a facility level requirement of different data elements
    """
    data_elements_requirement = pd.merge(report_requirement, data_element_list, left_on='dataset_ID', right_on='datasets_ID', 
                                     how='inner')
    data_elements_numbers = DataFrame(data_elements_requirement['data_element_ID'].value_counts())
    data_elements_numbers.columns = ['required_data_elements']
    return data_elements_numbers

Then compute how much orgunits actually report monthly

In [ ]:
def get_availability_orgunit(data):
    """
    Function that reports the number of data elements available for each orgunit by period 
    """
    data_availability = pd.pivot_table(data , index=['org_unit_ID','period'] , aggfunc = len)['data_element_ID']
    return data_availability

def get_availability_data_element(data):
    """
    Function that reports the number of data elements available for each data_element by period 
    """
    data_availability = pd.pivot_table(data , index=['data_element_ID','period'] , aggfunc = len)['org_unit_ID']
    return data_availability

In [ ]:
def get_completeness_orgunit(data , data_element_list , report_requirement):
    """
    Function that computes the data requirement and availability, and returns a description of completeness
    """
    required = get_requirement_orgunit(data_element_list , report_requirement)
    available = get_availability_orgunit(data)
    completeness = pd.merge(available.reset_index() , required.reset_index() , 
                            left_on = 'org_unit_ID' , right_on = 'index' ,
                            left_index=True, 
                            how='inner', sort=False)
    completeness['completeness'] = completeness['data_element_ID'] / completeness['required_data_elements']
    completeness = DataFrame(completeness)
    completeness = completeness[(completeness.completeness <= 1) & (completeness.period.astype(int) > 201012)]
    return completeness


def get_completeness_data_element(data , data_element_list , report_requirement):
    """
    Function that computes the data requirement and availability, and returns a description of completeness
    """
    required = get_requirement_data_element(data_element_list , report_requirement)
    print required.head()
    available = get_availability_data_element(data)
    completeness = pd.merge(available.reset_index() , required.reset_index() , 
                            left_on = 'data_element_ID' , right_on = 'index' ,
                            left_index=True, 
                            how='inner', sort=False)
    completeness['completeness'] = completeness['org_unit_ID'] / completeness['required_data_elements']
    
    completeness = DataFrame(completeness)
    completeness = completeness[(completeness.completeness <= 1) & (completeness.period.astype(int) > 201012)]
    return completeness

In [ ]:
print 'Making completeness for OU'
data_completeness_fac = get_completeness_orgunit(data_moh , data_elements_list , org_units_report)
print 'Making completeness for DE'
data_completeness_de = get_completeness_data_element(data_moh , data_elements_list , org_units_report)

In [ ]:
print data_completeness_de['required_data_elements'].value_counts()

In [ ]:
data_completeness_de2 = data_completeness_de[data_completeness_de["required_data_elements"]  < 8970]
print 100*len(data_completeness_de2)  / len(data_completeness_de)

In [ ]:
data_completeness_de2 = data_completeness_de[data_completeness_de["required_data_elements"]  < 7000]
data_completeness_de2 = data_completeness_de2.drop(['level_0' , 'org_unit_ID' , 'index' , 
                                                    'required_data_elements' ] , axis = 1)
#data_completeness_de2.reset_index(inplace=True) 
print len(data_completeness_de2)
print len(data_completeness_de)

In [ ]:
df = data_completeness_de2.set_index(['period' , 'data_element_ID']).unstack(['data_element_ID']) 
print df.plot(legend = False , figsize = [15,10])

In [ ]:
# How many values do we have ?
n_values_1 = len(data_moh_wide) - pd.isnull(data_moh_wide).sum(axis=0)
n_data_points = len(data_moh_wide)
print 'Maximum completeness for an indicator is' , (n_values_1 / n_data_points).max()
print 'Minimum completeness for an indicator is' , (n_values_1 / n_data_points).min()
print 'Mean completeness for an indicator is' , (n_values_1 / n_data_points).mean()
print 'Median completeness for an indicator is' , (n_values_1 / n_data_points).median()

CAVEAT : An unrlying hypothesis here is that every orgunit has to report on every data set every month. It is thus a raw UNDERESTIMATION of completeness

In [ ]:
data_moh_wide = data_moh_wide.unstack(1)

n_values_2 = len(data_moh_wide) - pd.isnull(data_moh_wide).sum(axis=0)

n_periods = data_moh.period.nunique()
print 'Maximum completeness for an indicator serie for a facility is' , (n_values_2 / n_periods).max()
print 'Minimum completeness for an indicator  for a facility is' , (n_values_2 / n_periods).min()
print 'Mean completeness for an indicator  for a facility is' , (n_values_2 / n_periods).mean()
print 'Median completeness for an indicator  for a facility is' , (n_values_2 / n_periods).median()

## Selection of Data

In [ ]:
# Here, we only want to keep facilities with at least one serie with over 75% completness, and series with at least 
# 50%  overall completeness

completeness_fac = DataFrame(n_values_2 / n_periods > 0.75)
completeness_de = DataFrame(n_values_1 / n_data_points > 0.5)

completeness_fac.reset_index(inplace=True) 
completeness_de.reset_index(inplace=True) 

de_keep = completeness_de[completeness_de[0] > 0.5].data_element_ID.unique()
fac_keep = completeness_fac[completeness_fac[0] > 0.5].org_unit_ID.unique()
print 'We want to keep' , 100 * len(fac_keep) / len(completeness_fac.org_unit_ID.unique()) , '% of facilities'
print 'We want to keep' , 100 * len(de_keep) / len(completeness_de.data_element_ID.unique()) , '% of indicators'

In [ ]:
data_comp = data_moh[data_moh['org_unit_ID'].isin(fac_keep) & data_moh['data_element_ID'].isin(de_keep)]
print 'We are keeping' , 100*len(data_comp) / len(data_moh) , '% of the data'

In [ ]:
#### And rebuilding the missingness

# First a very wide melt to get complete missing values in time
data_comp_wide = data_comp.set_index(['org_unit_ID', 'period', 
                                      'category','data_element_ID']).unstack(['org_unit_ID'  ,'data_element_ID',
                                                                                 'category']) 

# Then we stack on org_unit_ID, so we have all period for all org_unit, including empty ones
data_comp_wide = data_comp_wide.stack(level=['org_unit_ID'] , dropna = False)



## We need to rename columns. not the simplest now
data_comp_wide.columns = [' '.join(col).strip() for col in data_comp_wide.columns.values]
data_comp_wide.reset_index(inplace=True)

## AMELIA

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R rm(list = ls())

In [ ]:
%%R --input data_comp_wide

save.image('//ihme.washington.edu/IHME/HOME/grlurton/imputation/python_export.RData')

Ameliorer le travail R en exportant les donnees en Rdata, et reimportant les resultats.

Faire tourner en parrallele

RAJOUTER LES COVARIABLES SITES. => sinon on a des observations completement missing

en plus faire tourner d'abord sur le subset qui m'interesse et apres sur le reste 

## A venir

ARIMA / Kalman Filter

Make some mix

Catarina : amelia + arima : more or less similar

arima est bon pour les facilites avec donnes partielles + statifie
	? code 

amelia : everything in one go : on annual data
	set threshold : si facility reports < 60 : set as missing.
	find most important covariate

Autres méthodes : MICE et Missforest